# Discussion 07: SQL

In this discussion, we will be covering a review of basic SQL and exploring more advanced and interesting SQL queries and questions.

Optional SQL Reference: https://www.w3schools.com/sql/default.asp

In [ ]:
# !conda install -y psycopg2
# !conda install -y postgresql
# !pip install ipython-sql
# !pip install sqlalchemy

import os
import numpy as np
import pandas as pd
import sqlalchemy

%load_ext sql

In [ ]:
postgresql_uri = "postgres://{}:@localhost:5432/ds100".format(os.environ['USER'])
default_db = postgresql_uri

%sql $postgresql_uri

## Basic Table Creation, Manipulation, and Queries

### Creating or dropping a table
```sql
CREATE TABLE table_name (
    column1 datatype,
    column2 datatype,
    column3 datatype,
   ....
);
```
```sql
DROP TABLE IF EXISTS table_name;
```
### Adding rows 
```sql
INSERT INTO table_name
VALUES (value1, value2, value3, ...);
```
### Updating rows
```sql
UPDATE table_name
SET column1 = value1, column2 = value2, ...
WHERE condition;
```
### Deleting rows
```sql
DELETE FROM table_name
WHERE condition;
```

## SQL: Structured Query Language

The canonical SQL query block includes the following clauses, in the order they appear. Square brackets indicate optional clauses.
```sql
SELECT ...   
  FROM ...
[WHERE ...]
[GROUP BY ...]
[HAVING ...]
[ORDER BY ...]
[LIMIT ...];
```
Query blocks can reference **one or more** tables, and be nested in various ways.

### Reading a SQL Expression: Order of Evaluation
We've seen a bunch of different query clauses in lecture and lab, and done some mixing and matching. How do they fit together? The order of evaluation should be thought of like this:

1. The `FROM` and `WHERE` clauses are evaluated to compute selections and joins.
2. The `GROUP BY` and `HAVING` clauses are evaluated to for groups resulting from the previous step
3. The `SELECT` clause is evaluated, including any aggregates
4. The `ORDER BY` clause is evaluated
5. The `LIMIT` clause is used to cut off output production.


## Joins

### Different Types of SQL JOINs
Here are the different types of the JOINs in SQL:

1. (INNER) JOIN: Returns records that have matching values in both tables
2. LEFT (OUTER) JOIN: Return all records from the left table, and the matched records from the right table
3. RIGHT (OUTER) JOIN: Return all records from the right table, and the matched records from the left table
4. FULL (OUTER) JOIN: Return all records when there is a match in either left or right table

<img src="joins.png">

Note: You do not always have to use the `JOIN` keyword to join sql tables. The following are equivalent:

```sql
SELECT column1, column2
FROM table1, table2
WHERE table1.id = table2.id;
```
```sql
SELECT column1, column2
FROM table1 JOIN table2 
ON table1.id = table2.id;
```

### Example: Joining Three Tables

Suppose we have the following tables:

**Clowns**( cid integer, cname text, booth text )  
**Balloons**( bid integer, bshape text, bcolor text )  
**Catalog**( cid integer, bid integer, cost float )

The **Catalog** table contains prices for **Balloons** sold by different **Clowns** standing at certain booths in a fair.

**Question 1**  
How may we query for the top 10 most expensive shapes sold by Whompers LeFou?

```sql
SELECT bshape, cost
FROM Clowns, Balloons, Catalog 
WHERE Clowns.cid=Catalog.cid
    AND Balloons.bid=Catalog.bid
    AND cname='Whompers LeFou'
ORDER BY cost DESC
LIMIT 10;
```

**Question 2**  
How many different colors are available at each booth?
```sql
SELECT booth, COUNT(DISTINCT bcolor)
FROM Clowns, Balloons, Catalog
WHERE Clowns.cid=Catalog.cid
    AND Balloons.bid=Catalog.bid
GROUP BY booth
```

**Question 3**  
What is the average cost of a balloon at booths that offer more than 3 red shapes per clown? Each clown at the booth does not necessarily have to be selling more than 3 shapes.

```sql
SELECT booth, avg(cost)
FROM Clowns, Balloons, Catalog
WHERE Clowns.cid=Catalog.cid
    AND Balloons.cid=Catalog.cid
    AND bcolor='red'
GROUP BY booth
HAVING COUNT(DISTINCT bshape)/COUNT(DISTINCT Clowns.cid) > 3
```

## Transposing Matrices in SQL

Recall in lecture how we can represent a "matrix" relation as `my_matrix(r, c, v)`.

In [ ]:
%%sql
DROP TABLE IF EXISTS my_matrix;

CREATE TABLE my_matrix(r INTEGER, c INTEGER, val FLOAT, PRIMARY KEY(r, c));

INSERT INTO my_matrix
SELECT rows.r, columns.c, CEIL(6*RANDOM())::INTEGER AS v
  FROM generate_series(0,2) AS rows(r),
       generate_series(0,1) AS columns(c); 

In [ ]:
%%sql

SELECT *  
FROM my_matrix;

How can we transpose this matrix? Let's think about the implementation. 

First, we should think about the definition. Let $M$ denote a matrix and $[M]_{ij}$ be the element in the $i^{th}$ row and $j^{th}$ column. The matrix transpose is defined as $$[M^T]_{ij}=[M]_{ji}$$ This suggests that all we naively have to do is make a SELECT statement that swaps `row` and `col`.

In [ ]:
%%sql

SELECT M.c AS r, M.r AS c, val 
FROM my_matrix AS M;

## Matrix Multiplication w/ SQL

We already know how we may do matrix multiplication in numpy but how may we achieve this in SQL? 

Recall how two matrices are multiplied together: 
If A is an *n × m* matrix and B is an *m × p* matrix, their matrix product AB is an *n × p* matrix, in which the m entries across a row of A are multiplied with the m entries down a column of B and summed to produce an entry of AB.

In [ ]:
np.random.seed(43)

# normally distributed random numbers, mean 0 variance 1
my_matrixA = np.random.randint(1,6, (3,2)).astype('float')
my_matrixA

In [ ]:
my_matrixB = np.random.randint(1,6, (2,3)).astype('float')
my_matrixB

In [ ]:
# Dot Product of MatrixA and MatrixB
my_matrixA @ my_matrixB

Let's first save these as tables in pandas.

In [ ]:
(col_id, row_id) = np.meshgrid(np.arange(2), np.arange(3))

mat_a = pd.DataFrame(
    np.vstack([row_id.flatten().T, col_id.flatten(), my_matrixA.flatten()]).T, 
    columns=['r', 'c', 'v'])

mat_a

In [ ]:
(col_id, row_id) = np.meshgrid(np.arange(3), np.arange(2))

mat_b = pd.DataFrame(
    np.vstack([row_id.flatten().T, col_id.flatten(), my_matrixB.flatten()]).T, 
    columns=['r', 'c', 'v'])

mat_b

In [ ]:
engine = sqlalchemy.create_engine(postgresql_uri)
with engine.connect() as conn:
    conn.execute("DROP TABLE IF EXISTS mat_a")
    mat_a.to_sql("mat_a", conn, index=False)

    conn.execute("DROP TABLE IF EXISTS mat_b")
    mat_b.to_sql("mat_b", conn, index=False)

In [ ]:
%%sql
-- verify the existence and correctness of our tables
select * from mat_a;

### Implementation 
How can we multiply these matrices? Let's recall how any single entry in AB is produced. 

<img src="mat_mul.png" width="300px"> 

Remember the formula for entry $(i,j)$ of the product AB. It is the sum of $A(i,k)*B(k,j)$ for all k. Thus, the `JOIN` condition `A.c = B.r` makes sure that both `A.v` and `B.v` has the *same k*. The `GROUP BY` clause makes sure that we sum over all k's.


In [ ]:
%%sql 

SELECT A.r::INTEGER,
   B.c::INTEGER,
   SUM(A.v * B.v) AS value
FROM mat_a A
INNER JOIN mat_b B
ON A.c = B.r
GROUP BY A.r, B.c;